Weighting sentences by their position in the review:

We calculate feature importance off of the test set's features because only the test set's sentences get scored (probssentences). Is this legal?

In [16]:
import pickle
import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [2]:
probssentences = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/probssentences.p', 'rb'))
test1 = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/test1.p', 'rb'))

In [3]:
#append doc value to test1
test1['doc'] = [ i for i in range(len(test1))]
print test1.shape
print probssentences.shape

(12500, 4)
(133515, 3)


In [5]:
test1[0:10]

,id,sentiment,review,doc
21130,"""5706_2""",0,"""This movie is like the thousand \""cat and mou...",0
3404,"""11132_3""",0,"""A documentarist, like any filmmaker, must con...",1
14560,"""10770_7""",1,"""With a well thought out cast, this movie was ...",2
6671,"""1058_10""",1,"""this is the best sci-fi that I have seen in m...",3
10982,"""12239_1""",0,"""Ever since I was eight years old I have been ...",4
5526,"""966_7""",1,"""2005 Toronto Film Festival Report It is offic...",5
19301,"""11645_2""",0,"""This is a review of 'Freddy Mercury The Untol...",6
17060,"""5920_9""",1,"""American icon Henry Fonda portrays \""Elegant\...",7
24744,"""9501_2""",0,"""This is the page for \""House of Exorcism\"", b...",8
5253,"""7877_4""",0,"""The makers of this film have created a future...",9


In [4]:
#append the true class value
probssentences["truelabel"] = 0
for i in range(len(probssentences)):
    #get true label value from test1 where index matches doc number
    j = probssentences.iloc[i]['doc']
    probssentences.set_value(i, 'truelabel', test1.iloc[[j]]["sentiment"])

In [5]:
probssentences[0:10]

,0,1,doc,truelabel
0,0.5986,0.4014,0,0
1,0.00792023,0.99208,0,0
2,0.369248,0.630752,0,0
3,0.445318,0.554682,0,0
4,0.999581,0.000419095,0,0
5,0.00449226,0.995508,1,0
6,0.918007,0.0819928,1,0
7,0.55223,0.44777,1,0
8,0.986808,0.0131918,1,0
9,0.721142,0.278858,1,0


In [6]:
#split test set into two
fracTrain = 0.5
nSamples = test1.shape[0]
print nSamples
order = np.random.permutation(nSamples) # come up with a random ordering
print len(order)
splitIndex = int(np.round(nSamples*fracTrain))
print len(order[:splitIndex])
print len(order[splitIndex:])
testA = test1.iloc[order[:splitIndex],:]
testB = test1.iloc[order[splitIndex:],:]

12500
12500
6250
6250


In [7]:
#split probssentences the same way
probssentencesA = probssentences.loc[probssentences['doc'].isin(testA['doc'].unique())]
probssentencesB = probssentences.loc[probssentences['doc'].isin(testB['doc'].unique())]

In [8]:
print len(probssentences)
print len(probssentencesA)
print len(probssentencesB)
print len(testA)
print len(testB)

133515
66494
67021
6250
6250


In [9]:
probssentencesA[0:10]

,0,1,doc,truelabel
0,0.5986,0.4014,0,0
1,0.00792023,0.99208,0,0
2,0.369248,0.630752,0,0
3,0.445318,0.554682,0,0
4,0.999581,0.000419095,0,0
5,0.00449226,0.995508,1,0
6,0.918007,0.0819928,1,0
7,0.55223,0.44777,1,0
8,0.986808,0.0131918,1,0
9,0.721142,0.278858,1,0


In [10]:
probssentencesB[0:10]

,0,1,doc,truelabel
12,0.0193013,0.980699,2,1
13,0.0159692,0.984031,2,1
14,0.0224181,0.977582,2,1
15,0.986999,0.0130013,2,1
35,3.08829e-07,1,5,1
36,0.139833,0.860167,5,1
37,0.193762,0.806238,5,1
38,0.666227,0.333773,5,1
39,0.0583559,0.941644,5,1
40,0.000877276,0.999123,5,1


In [11]:
testA[0:10]

,id,sentiment,review,doc
10023,"""8251_3""",0,"""Watching TRUTH ABOUT LOVE (is this a double e...",12489
3017,"""4539_9""",1,"""The acting is pretty cheesy, but for the peop...",8874
4273,"""2824_2""",0,"""I'm a big Porsche fan, and the car was the be...",10858
496,"""654_10""",1,"""In what could have been seen as a coup toward...",1077
2399,"""10395_2""",0,"""Quite what the producers of this appalling ad...",1194
13924,"""5842_8""",1,"""Can I be as simple and primitive in my evalua...",5439
5442,"""9275_9""",1,"""Panahi, whose previous films such as The Circ...",9356
8579,"""10049_8""",1,"""I am not a parent, neither am I a male. But I...",4557
4356,"""2576_3""",0,"""THE WATERDANCE (1991) The main character of T...",8210
6960,"""4870_4""",0,"""The movie is bad. Nothing special - just a \""...",10340


In [13]:
#checking that testB matches probssentencesB
print testB.loc[testB['doc']==2]

              id  sentiment  \
14560  "10770_7"          1   

                                                  review  doc  
14560  "With a well thought out cast, this movie was ...    2  


### NOTE: probssentences and test set are not in same order. Must match on doc number.

In [14]:
print len(testA['doc'].unique())

6250


In [32]:
a = datetime.datetime.now().replace(microsecond=0)

#put into function and make split number the parameter
def getFeaturesAndLabels(numSplits):
    i = 0
    features = []
    labels = []
    #for each doc in probssentences:
    for doc in testA['doc'].unique():  
        avgScores = []
        df = probssentencesA.loc[probssentencesA['doc'] == doc]
        if len(df) >= numSplits:
            #split into x groups
            lst = np.array_split(df[[0]], numSplits)
            for z in lst:
                #make the average score for each group the feature
                avgScores.append(z.mean()[0])
                #convert to a 0 or 1 (?)
            features.append(avgScores)
            labels.append(testA.loc[testA['doc'] == doc]['sentiment'].values[0])
        i = i + 1
    return (features, labels)
    
#function call
x = 3
tuple1 = getFeaturesAndLabels(x)

b = datetime.datetime.now().replace(microsecond=0)
print b-a

0:00:57


In [35]:
#ignore those reviews with fewer sentences than splits, so instead of 6250 datapoints to build on, have lost about 242:
print len(tuple1[0])
print len(tuple1[1])
features = tuple1[0]
labels = tuple1[1]

6008
6008


In [36]:
#perform feature weighting-
clf = ExtraTreesClassifier()
clf = clf.fit(features, labels)
weights = clf.feature_importances_  
print weights

[ 0.33510705  0.28484426  0.38004869]


### Split test set 2 into x subsets as well

In [33]:
a = datetime.datetime.now().replace(microsecond=0)

#put into function and make split number the parameter
x = 3

i = 0
featuresTest = []
labelsTest = []
#for each doc in probssentences:
for doc in testB['doc'].unique():  
    avgScoresTest = []
    df = probssentencesB.loc[probssentencesB['doc'] == doc]
    if len(df) >= x:
        #split into x groups
        lst = np.array_split(df[[0]], x)
        for z in lst:
            #make the average score for each group the feature
            avgScoresTest.append(z.mean()[0])
            #convert to a 0 or 1 (?)
        featuresTest.append(avgScoresTest)
        labelsTest.append(testB.loc[testB['doc'] == doc]['sentiment'].values[0])
    i = i + 1

b = datetime.datetime.now().replace(microsecond=0)
print b-a

0:00:59


In [45]:
print (featuresTest*weights).shape
print featuresTest[0]

(6015, 3)
[0.99864677588144934, 0.97593297561009729, 0.75025797883669532]


In [ ]:
print 

In [39]:
preds = np.ones((prob.shape[0]))
print preds.shape

(12500,)


In [40]:
preds[np.where(prob.iloc[:,0] >  prob.iloc[:,1])] = 0 # The first column is the negative model

In [41]:
print preds.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [42]:
print np.size(np.where(preds == test1["sentiment"]))*1./np.size(preds)

0.86848


In [15]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(test1["sentiment"], preds)

0.868160030313


Especially considering the weights are fit from the test data itself, we would expect to see a big increase in performance, which we're not getting.

Why? :/ could have something to do with the fact that we fit a classifier on the test data which is pretty wacky. Or could be the way the weights were applied to the sentence scores (unlikely). 